In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from tensorflow.keras.optimizers import SGD,RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.regularizers import l1


from scipy import stats
from scipy.stats import norm, skew, pearsonr
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [0]:
gk = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/gk_final.pkl").drop_duplicates(ignore_index = True, keep = False)
defenders = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/defenders_final.pkl").drop_duplicates(ignore_index = True, keep = False)
midfielders = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/midfielders_final.pkl").drop_duplicates(ignore_index = True, keep = False)
attackers = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/attackers_final.pkl").drop_duplicates(ignore_index = True, keep = False)

In [0]:
gk["type"] = gk["type"].astype('object')
defenders["type"] = defenders["type"].astype('object')
midfielders["type"] = midfielders["type"].astype('object')
attackers["type"] = attackers["type"].astype('object')

gk.replace({"mid_seson": "mid_season"}, inplace = True)
defenders.replace({"mid_seson": "mid_season"}, inplace = True)
midfielders.replace({"mid_seson": "mid_season"}, inplace = True)
attackers.replace({"mid_seson": "mid_season"}, inplace = True)

# Data Preperation

## Scaling the output variable

In [0]:
gk['mv_log'] = np.log1p(gk["mv"])

defenders['mv_log'] = np.log1p(defenders["mv"])

midfielders['mv_log'] = np.log1p(midfielders["mv"])

attackers['mv_log'] = np.log1p(attackers["mv"])

In [0]:
def one_hot(df, cat_list):
    encoder = OneHotEncoder()
    df_encoded = encoder.fit_transform(df[cat_list]).toarray()
    
    cols = encoder.get_feature_names()
    encoded_df = pd.DataFrame(df_encoded, columns = cols, dtype = "int64")
    df.drop(cat_list, axis = 1, inplace = True)
    
    df_final = df.join(encoded_df, on = df.index)
    
    return df_final

def remove_outliers(df, threshold):
    z_score = np.abs(stats.zscore(df))
    df_rem_out = df[(z_score < threshold).all(axis=1)]
    df_rem_out.reset_index(inplace = True, drop = True)
    return df_rem_out

def prepare_data(df,drop_cols,cat_cols,rm_out = True, output = "fee_log",drop_cat = False):
  df_dropped = df.drop(drop_cols, axis = 1)
  y = df_dropped[output]
  X = df_dropped.drop(output,axis = 1)
  X = remove_corr(X)
  not_scaled = cat_cols + ["tourn_year"] 
  scaling = lambda col : (col - col.mean())/(col - col.std()) if col.name not in not_scaled else col
  X = X.apply(scaling)
  if(drop_cat):
    X = X.drop(cat_cols, axis = 1)
  else:
    X = one_hot(X, cat_cols)
  if(rm_out):
    df_merged = pd.concat([X,y], axis = 1)
    df_no_out = remove_outliers(df_merged,3)
    X = df_no_out.drop('fee_log', axis = 1)
    y = df_no_out["fee_log"]
  return (X,y)

def fsel_rforest(X_train, y_train, n):
  assert n < X_train.shape[1]
  rfr = RandomForestRegressor(max_depth = 18)
  rfr.fit(X_train, y_train)
  df = pd.DataFrame({'feature':X_train.columns,'importance':rfr.feature_importances_})
  df.sort_values(by = 'importance', inplace = True, ascending = False)
  return (df.iloc[0:n,0:1]["feature"].unique())

def remove_corr(df, threshold = 0.6):
  corr_matrix = df.corr().abs()
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
  to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
  to_drop = list(filter(lambda x: x != 'mv_log', to_drop))
  print(to_drop)
  return df.drop(df[to_drop], axis = 1)

## One hot encoding,scaling and feature selection

In [0]:
gk_drop_cols = ["tm_id", "sf", "from", "to", "transfer_season",
               "loan", "main_field_position", "field_position", 
               "dob", "stats_season", "year", "date", "cum_mv","mv","fee"]

gk_cat_cols = ["type", "continent"]

cat_cols = ["type", "continent", "field_position", "sf"]

def_drop_cols = ["tm_id", "transfer_season",
                "main_field_position", 
                "stats_season", "fee", "mv", "year"]

mid_drop_cols = ["tm_id", "transfer_season",
                "main_field_position", 
                "stats_season", "fee", "mv", "year"]


att_drop_cols = ["tm_id", "transfer_season",
               "main_field_position", 
                "stats_season", "fee", "mv", "year"]

In [0]:
gk_X,gk_y = prepare_data(gk,gk_drop_cols,gk_cat_cols,drop_cat = True)
gk_X_train,gk_X_test,gk_y_train,gk_y_test = train_test_split(gk_X,gk_y,test_size=0.2,random_state=1)

['cum_winner', 'cs', 'gc', 'cum_cs', 'cum_gc', 'cum_ppg', 'fwpct', 'cum_gcpcs', 'cum_fwpct']


In [0]:
gk_cols = fsel_rforest(gk_X_train, gk_y_train, 5)
gk_X_train = gk_X_train[gk_cols]; gk_X_test = gk_X_test[gk_cols]

In [0]:
def_X,def_y = prepare_data(defenders,def_drop_cols,cat_cols,drop_cat = True)
def_X_train,def_X_test,def_y_train,def_y_test = train_test_split(def_X,def_y,test_size=0.1,random_state=1)

['mp', 'fwpct', 'cum_a', 'cum_g', 'cum_yc', 'cum_rc', 'cum_fwpct', 'cum_winner']


In [0]:
def_cols = fsel_rforest(def_X_train, def_y_train, 5)
def_X_train = def_X_train[def_cols]; def_X_test = def_X_test[def_cols]

In [0]:
mid_X,mid_y = prepare_data(midfielders,mid_drop_cols,cat_cols,drop_cat = True)
mid_X_train,mid_X_test,mid_y_train,mid_y_test = train_test_split(mid_X,mid_y,test_size=0.1,random_state=1)

['mp', 'fmpct', 'fwpct', 'cum_a', 'cum_g', 'cum_yc', 'cum_fwpct', 'cum_winner']


In [0]:
mid_cols = fsel_rforest(mid_X_train, mid_y_train, 5)
mid_X_train = mid_X_train[mid_cols]; mid_X_test = mid_X_test[mid_cols]

In [0]:
att_X,att_y = prepare_data(attackers,att_drop_cols,cat_cols,drop_cat = True)
att_X_train,att_X_test,att_y_train,att_y_test = train_test_split(att_X,att_y,test_size=0.1,random_state=1)

['app', 'mp', 'gppg', 'fmpct', 'fwpct', 'cum_g', 'cum_fwpct', 'cum_winner']


In [0]:
att_cols = fsel_rforest(att_X_train, att_y_train, 5)
att_X_train = att_X_train[att_cols]; att_X_test = att_X_test[att_cols]

In [0]:
gk_model = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/gk_nn.h5"
def_model = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/def_nn.h5"
mid_model = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/mid_nn.h5"
att_model = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/att_nn.h5"

In [0]:
gk_history = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/gk_history.pkl"
def_history = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/def_history.pkl"
mid_history = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/mid_history.pkl"
att_history = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/att_history.pkl"

# Visualization functions

In [0]:
chart_title = "Player's actual and predicted transfer price"

def chart_regression(pred,y,scaled_back = False, sort_y = False, title = chart_title, y_axis = "Player's transfer price"):
  if(scaled_back):
    pred = np.expm1(pred)
    y = np.expm1(y)
  random_x = np.linspace(0, 1, 100)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=random_x, y=pred,
                    mode='lines',
                    name='predicted'))
  fig.add_trace(go.Scatter(x=random_x, y=y,
                  mode='lines',
                  name='actual'))
  fig.update_layout(title= title,
                    yaxis_title=y_axis)
  fig.show()

loss_title = "Model's training and validation loss over epochs"

def loss_chart(history, rmse = False, title = loss_title, y_axis = "Mean Squared Error"):
  epochs = np.arange(1,len(history['loss']) + 1, 1)
  t_loss = history['loss']
  v_loss = history['val_loss']
  if(rmse):
    t_loss = np.sqrt(t_loss)
    v_loss = np.sqrt(v_loss)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=epochs, y=t_loss,
                    mode='lines',
                    name='training loss'))
  fig.add_trace(go.Scatter(x=epochs, y=v_loss,
                  mode='lines',
                  name='validation loss'))
  fig.update_layout(title=title,
                    xaxis_title="Number of epochs",
                    yaxis_title = y_axis
                    )
  fig.show()

scatter_title = "Actual transfer price vs predicted transfer price by positions"

def scatter(df, x, y, filter_pos = False, filter_var = "position",scaled_back = True,
            filter_res = None, facet = True, title = scatter_title, 
            x_label = "Predicted Price", y_label = "Actual Price"):
  if(scaled_back):
    df[x] = np.expm1(df[x])
    df[y] = np.expm1(df[y])
  if(filter_pos):
    df = df[df[filter_var] == filter_res]
  if(facet):
    fig = px.scatter(df, x, y, labels = {x: x_label, y: y_label }, title = title,
                     facet_col = "position", facet_col_wrap = 2, color = "position")
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
    fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  else:
    fig = px.scatter(df, x, y, labels = {x: x_label, y: y_label }, title = title)
  corr, _ = pearsonr(df[x],df[y])
  fig.show()
  print("Correlation between the variables",corr)
  

# Building neural networks

In [0]:
def flexible_nn(dim, X_train, y_train, epochs = 2000):
  model = Sequential()
  model.add(Dense(100,activation='relu',activity_regularizer = l1(0.03), 
                  kernel_regularizer=l1(0.03), bias_regularizer=l1(0.03))) 
  model.add(Dense(50,activation='relu',activity_regularizer = l1(0.03), 
                  kernel_regularizer=l1(0.03), bias_regularizer=l1(0.03)))
  model.add(Dense(25,activation='relu',activity_regularizer = l1(0.03),
                  kernel_regularizer=l1(0.03), bias_regularizer=l1(0.03)))
  model.add(Dense(10,activation='relu',activity_regularizer = l1(0.03), 
                  kernel_regularizer=l1(0.03), bias_regularizer=l1(0.03)))
  model.add(Dense(1)) 
  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=0.05, 
        patience=epochs/10, verbose=1, mode='auto',
        restore_best_weights=True)
  history = model.fit(X_train,y_train,validation_split = 0.2 ,verbose=2,
            callbacks = [monitor],epochs=epochs, batch_size = 150)
  return (model,history)

In [0]:
def simple_nn(dim, X_train, y_train, X_test, y_test, epochs = 2000):
  model = Sequential()
  model.add(Dense(25,input_dim = dim, activation = 'relu',activity_regularizer = l1(0.03),
                  kernel_regularizer = l1(0.03), bias_regularizer = l1(0.03)))
  model.add(Dropout(0.5)) 
  model.add(Dense(10, activation = 'relu', activity_regularizer = l1(0.03),
                  kernel_regularizer = l1(0.03), bias_regularizer = l1(0.03)))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=0.05, 
      patience=epochs/10, verbose=1, mode='auto',
      restore_best_weights=True)
  history = model.fit(X_train,y_train,validation_data=(X_test,y_test),verbose=2,
            callbacks = [monitor],epochs=epochs)
  return (model,history)


# GK

## Training

In [0]:
gk_model,gk_history = simple_nn(gk_X_train.shape[1],gk_X_train,gk_y_train,gk_X_test,gk_y_test)

Epoch 1/2000
10/10 - 0s - loss: 209.3450 - val_loss: 200.3369
Epoch 2/2000
10/10 - 0s - loss: 194.8259 - val_loss: 192.4047
Epoch 3/2000
10/10 - 0s - loss: 196.2683 - val_loss: 184.3011
Epoch 4/2000
10/10 - 0s - loss: 182.5498 - val_loss: 176.3773
Epoch 5/2000
10/10 - 0s - loss: 177.7176 - val_loss: 168.3687
Epoch 6/2000
10/10 - 0s - loss: 164.8048 - val_loss: 160.3232
Epoch 7/2000
10/10 - 0s - loss: 162.5848 - val_loss: 153.1486
Epoch 8/2000
10/10 - 0s - loss: 150.4895 - val_loss: 145.1572
Epoch 9/2000
10/10 - 0s - loss: 146.0663 - val_loss: 137.2897
Epoch 10/2000
10/10 - 0s - loss: 138.7312 - val_loss: 130.4007
Epoch 11/2000
10/10 - 0s - loss: 135.3520 - val_loss: 123.9549
Epoch 12/2000
10/10 - 0s - loss: 130.8879 - val_loss: 118.7561
Epoch 13/2000
10/10 - 0s - loss: 131.0648 - val_loss: 114.2238
Epoch 14/2000
10/10 - 0s - loss: 126.8837 - val_loss: 110.5328
Epoch 15/2000
10/10 - 0s - loss: 115.8283 - val_loss: 105.4060
Epoch 16/2000
10/10 - 0s - loss: 122.9064 - val_loss: 100.9522
E

## Scoring

In [0]:
gk_pred = gk_model.predict(gk_X_test)
gk_score = np.sqrt(metrics.mean_squared_error(gk_y_test,gk_pred))
gk_actual_score = np.sqrt(metrics.mean_squared_error(np.expm1(gk_y_test), np.expm1(gk_pred)))
print("Final score in log scale (RMSE): {}".format(gk_score))
print("Final score in numeric scale (RMSE): {}".format(gk_actual_score))

Final score in log scale (RMSE): 0.9845355049495835
Final score in numeric scale (RMSE): 3280626.73505736


## Visualize predictions

In [0]:
chart_regression(gk_pred.flatten(),gk_y_test, scaled_back = True)

In [0]:
loss_chart(pd.DataFrame(gk_history.history), title = "Model's training and validation loss over epochs(GK)",)

# Defenders

## Training

In [0]:
def_model,def_history = flexible_nn(def_X_train.shape[1],def_X_train,def_y_train)

Epoch 1/2000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

9/9 - 0s - loss: 227.7687 - val_loss: 225.2402
Epoch 2/2000
9/9 - 0s - loss: 223.8286 - val_loss: 221.4471
Epoch 3/2000
9/9 - 0s - loss: 217.8556 - val_loss: 214.3047
Epoch 4/2000
9/9 - 0s - loss: 207.2185 - val_loss: 201.4304
Epoch 5/2000
9/9 - 0s - loss: 190.2057 - val_loss: 182.0283
Epoch 6/2000
9/9 - 0s - loss: 176.3748 - val_loss: 165.2717
Epoch 7/2000
9/9 - 0s - loss: 169.2632 - val_loss: 156.3294
Epoch 8/2000
9/9 - 0s - loss: 159.2225 - val_loss: 149.5917
Epoch 9/2000
9/9 - 0s - loss

## Scoring

In [0]:
def_pred = def_model.predict(def_X_test)
  
def_score = np.sqrt(metrics.mean_squared_error(def_y_test,def_pred))
def_actual_score = np.sqrt(metrics.mean_squared_error(np.expm1(def_y_test), np.expm1(def_pred)))
print("Final score in log scale (RMSE): {}".format(def_score))
print("Final score in numeric scale (RMSE): {}".format(def_actual_score))

Final score in log scale (RMSE): 0.8828811655312204
Final score in numeric scale (RMSE): 3539680.179998761


## Visualize predictions

In [0]:
chart_regression((def_pred.flatten()),def_y_test)

In [0]:
loss_chart(pd.DataFrame(def_history.history),title = "Model's training and validation loss over epochs(Defenders)") 

# Midfielders

## Training

In [0]:
mid_model,mid_history = flexible_nn(mid_X_train.shape[1],mid_X_train,mid_y_train)

Epoch 1/2000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

9/9 - 0s - loss: 229.6623 - val_loss: 220.9780
Epoch 2/2000
9/9 - 0s - loss: 217.8631 - val_loss: 207.8227
Epoch 3/2000
9/9 - 0s - loss: 203.4157 - val_loss: 190.6782
Epoch 4/2000
9/9 - 0s - loss: 184.5392 - val_loss: 170.5319
Epoch 5/2000
9/9 - 0s - loss: 163.6733 - val_loss: 143.6608
Epoch 6/2000
9/9 - 0s - loss: 147.7696 - val_loss: 123.8460
Epoch 7/2000
9/9 - 0s - loss: 137.5815 - val_loss: 113.1613
Epoch 8/2000
9/9 - 0s - loss: 129.7990 - val_loss: 108.8175
Epoch 9/2000
9/9 - 0s - loss

## Scoring

In [0]:
mid_pred = mid_model.predict(mid_X_test)
  
mid_score = np.sqrt(metrics.mean_squared_error(mid_y_test,mid_pred))
mid_actual_score = np.sqrt(metrics.mean_squared_error(np.expm1(mid_y_test), np.expm1(mid_pred)))
print("Final score in log scale (RMSE): {}".format(mid_score))
print("Final score in numeric scale (RMSE): {}".format(mid_actual_score))

Final score in log scale (RMSE): 0.869854106851569
Final score in numeric scale (RMSE): 5912809.736429171


## Visualize predictions

In [0]:
chart_regression(mid_pred.flatten(),mid_y_test, scaled_back = True)

In [0]:
loss_chart(pd.DataFrame(mid_history.history), title = "Model's training and validation loss over epochs(Midfielders)") 

# Attackers

## Training

In [0]:
att_model,att_history = flexible_nn(att_X_train.shape[1],att_X_train,att_y_train)

Epoch 1/2000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11/11 - 0s - loss: 238.0386 - val_loss: 227.9122
Epoch 2/2000
11/11 - 0s - loss: 229.7700 - val_loss: 221.1591
Epoch 3/2000
11/11 - 0s - loss: 221.9498 - val_loss: 211.2118
Epoch 4/2000
11/11 - 0s - loss: 207.6586 - val_loss: 198.6459
Epoch 5/2000
11/11 - 0s - loss: 184.6356 - val_loss: 203.1141
Epoch 6/2000
11/11 - 0s - loss: 158.9619 - val_loss: 225.5434
Epoch 7/2000
11/11 - 0s - loss: 134.4134 - val_loss: 211.9171
Epoch 8/2000
11/11 - 0s - loss: 112.0444 - val_loss: 179.9266
Epoch 9/2000

## Scoring

In [0]:
att_pred = att_model.predict(att_X_test)
  
att_score = np.sqrt(metrics.mean_squared_error(att_y_test,att_pred))
att_actual_score = np.sqrt(metrics.mean_squared_error(np.expm1(att_y_test), np.expm1(att_pred)))
print("Final score in log scale (RMSE): {}".format(att_score))
print("Final score in numeric scale (RMSE): {}".format(att_actual_score))

Final score in log scale (RMSE): 0.8165060494516995
Final score in numeric scale (RMSE): 4832662.185791148


## Visualize predictions

In [61]:
chart_regression((att_pred.flatten()),att_y_test)

In [0]:
loss_chart(pd.DataFrame(att_history.history),title = "Model's training and validation loss over epochs(Attackers)") 

# Actual Price vs Predicted Price

In [0]:
results = pd.DataFrame({'actual':gk_y_test, 'predicted':gk_pred.flatten(), 'position':"Goalkeepers"})
results = pd.concat([results,pd.DataFrame({'actual':def_y_test, 'predicted':def_pred.flatten(), 'position':"Defenders"})])
results = pd.concat([results,pd.DataFrame({'actual':mid_y_test, 'predicted':mid_pred.flatten(), 'position':"Midfielders"})])
results = pd.concat([results,pd.DataFrame({'actual':att_y_test, 'predicted':att_pred.flatten(), 'position':"Attackers"})])
results.reset_index(inplace = True)

In [0]:
scatter(results, 'predicted', 'actual', scaled_back = True)

Correlation between the variables 0.9030126991219999


In [0]:
rmse_df = pd.DataFrame([int(gk_actual_score), int(def_actual_score), int(mid_actual_score), int(att_actual_score)],
             ['Goalkeepers',"Defenders","Midfielders","Attackers"]).reset_index()
rmse_df.columns = ['position','score']
rmse_df.sort_values(by = "score", ascending = True, inplace = True)

In [68]:
fig_ft = px.bar(rmse_df, x = 'score', y = 'position',
       orientation = 'h', color = 'position', 
       title = 'RMSE on each position', 
       labels = {'score': 'RMSE(in millions)', 'position': 'Position'})
fig_ft.update_layout(showlegend=False)
fig_ft.show()

In [76]:
corr, _ = pearsonr(results[results.position == "Goalkeepers"]['predicted'],results[results.position == "Goalkeepers"]['actual'])
corr

0.7770989437110722

In [77]:
corr, _ = pearsonr(results[results.position == "Defenders"]['predicted'],results[results.position == "Defenders"]['actual'])
corr

0.8701851018125166

In [78]:
corr, _ = pearsonr(results[results.position == "Midfielders"]['predicted'],results[results.position == "Midfielders"]['actual'])
corr

0.9122575568438047

In [79]:
corr, _ = pearsonr(results[results.position == "Attackers"]['predicted'],results[results.position == "Attackers"]['actual'])
corr

0.9141439580382155

In [0]:
# results.to_pickle("/content/drive/My Drive/Capstone/Data/Summary/Deep_Learning/nn_results.pkl")
# results.to_pickle("./nn_results.pkl")

# Save models

In [0]:
import os
save_path = "/content/drive/My Drive/Capstone/Saved Models/Deep Learning/"
gk_model.save(os.path.join(save_path,"gk_nn.h5"))
def_model.save(os.path.join(save_path,"def_nn.h5"))
mid_model.save(os.path.join(save_path,"mid_nn.h5"))
att_model.save(os.path.join(save_path,"att_nn.h5"))

In [0]:
pd.DataFrame(gk_history.history).to_pickle((os.path.join(save_path,"gk_history.pkl")))
pd.DataFrame(def_history.history).to_pickle((os.path.join(save_path,"def_history.pkl")))
pd.DataFrame(mid_history.history).to_pickle((os.path.join(save_path,"mid_history.pkl")))
pd.DataFrame(att_history.history).to_pickle((os.path.join(save_path,"att_history.pkl")))

In [0]:
from tensorflow.keras.models import load_model
att_loaded_model = load_model(os.path.join(save_path,"att_nn.h5"))
att_history = pd.read_pickle((os.path.join(save_path,"att_history.pkl")))

In [0]:
loss_chart(att_history)